In [6]:
from pyreadr import read_r


from arboreto.algo import grnboost2
from arboreto.utils import load_tf_names
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase

import pandas as pd
import os

### Prelimanary

Load count matrix and TFs names. \
\
Note: 
<ul>
    <li>count matrix generated by R script import_data.R, from file data/dpn.vnc.domain.labelled.rds </li>
    <li>TF names are copied from https://github.com/aertslab/pySCENIC/blob/master/resources/allTFs_dmel.txt (29.3.2023)</li>
<ul>

In [7]:
ex_matrix = pd.read_csv("../data/expression_mat.csv", index_col=0)  # load count matrix 
tf_names = load_tf_names("../data/allTFs_dmel.txt") # Derive list of Transcription Factors(TF) for Drosophila

In [8]:
ex_matrix.shape

(4672, 9751)

In [9]:
ex_matrix

,a,abd-A,Abd-B,Abl,abo,ac,acj6,Acph-1,Act5C,Act42A,...,lncRNA:CR43716,lncRNA:CR44997,asRNA:CR45151,lncRNA:CR45310,lncRNA:CR45425,asRNA:CR45822,asRNA:CR45891,lncRNA:CR45961,lncRNA:CR46032,lncRNA:CR46119
TP1_AACTCAGGTAAATACG,0.0,0.000000,0.0,0.0,0.466624,0.000000,0.0,0.0,2.764117,1.750383,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
TP1_TCACGAATCTATCGCC,0.0,1.053208,0.0,0.0,0.000000,0.483831,0.0,0.0,2.879282,1.053208,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
TP1_TACGGTACAATAGAGT,0.0,0.813928,0.0,0.0,0.000000,0.000000,0.0,0.0,2.458085,1.059571,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
TP1_CAACTAGAGAGACGAA,0.0,0.000000,0.0,0.0,0.000000,0.888313,0.0,0.0,2.728522,2.006819,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
TP1_GCGCCAAAGTCGATAA,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,3.294131,1.989672,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TP2.2_TCTTTCCGTACCGTTA,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,3.506908,2.466545,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
TP2.2_AGCCTAATCTGCAAGG,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,3.109142,2.836230,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
TP2.2_GATTCAGCACACGCTG,0.0,2.470005,0.0,0.0,0.000000,1.858055,0.0,0.0,3.119940,1.858055,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
TP2.2_TGAGCATGTGATGTCT,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.854410,2.466052,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0


Load ranking databases (for motif enrichment)  --> https://resources.aertslab.org/cistarget/ (https://resources.aertslab.org/cistarget/databases/drosophila_melanogaster/dm6/flybase_r6.02/mc8nr/gene_based/)

In [10]:
db_fnames = "../data/dm6-5kb-upstream-full-tx-11species.mc8nr.genes_vs_motifs.rankings.feather"
dbs = [RankingDatabase(fname=db_fnames, name=os.path.splitext(os.path.basename(db_fnames))[0])]
dbs


[FeatherRankingDatabase(name="dm6-5kb-upstream-full-tx-11species.mc8nr.genes_vs_motifs.rankings")]

Motif annotation file: https://resources.aertslab.org/cistarget/motif2tf/ (v8 matches mc8nr db)

In [11]:
motif_annotation_file = "../data/motifs-v8-nr.flybase-m0.001-o0.0.tbl"

#### Run Pipeline
line by line to get familiar with the outputs, and time the commands. 

In [12]:
# sample matrix:
ex_matrix_sub = ex_matrix.iloc[1:100,1:500]
# run first step
adjacencies = grnboost2(ex_matrix_sub, tf_names, verbose=True)

preparing dask client
parsing input
creating dask graph
4 partitions
computing dask graph
shutting down client and local cluster
finished


In [13]:
adjacencies

,TF,target,importance
53,Brd,E(spl)m5-HLH,3.687917e+01
495,sna,sca,2.138833e+01
465,sala,sca,1.828094e+01
333,E(spl)m5-HLH,rst,1.794850e+01
172,FER,Gart,1.668760e+01
...,...,...,...
218,hay,Arl1,3.657316e-16
248,ImpL2,nkd,2.747921e-16
22,AnxB9,ase,2.530264e-16
132,ec,LanA,1.584127e-16


In [14]:
modules = list(modules_from_adjacencies(adjacencies, ex_matrix_sub)) 
modules


2023-04-19 10:09:33,870 - pyscenic.utils - INFO - Calculating Pearson correlations.

2023-04-19 10:09:33,880 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [False].

2023-04-19 10:09:34,058 - pyscenic.utils - INFO - Creating modules.


[Regulon(name='Regulon for Act42A', gene2weight=frozendict.frozendict({'H': 1.276060387537715, 'E(spl)malpha-BFM': 1.2280484203292, 'Fib': 1.1603922584011095, 'PpV': 1.1435658981409325, 'dsh': 1.049795923406892, 'kni': 0.9166891474718655, 'fj': 0.8515994941804916, 'Dr': 0.8205145254213921, 'peb': 0.7678413491730193, 'RpII215': 0.7638053980319702, 'e(y)1': 0.717069228859746, 'CycA': 0.6641494754003411, 'Rm62': 0.6436636333839109, 'Bsg25D': 0.6113942249256985, 'Myb': 0.5982878251309913, 'mod': 0.5739044861205753, 'Kr': 0.5559288253820063, 'slp1': 0.5236419567192246, 'Hsc70-1': 0.5023176443274258, 'Pc': 0.4922311246979344, 'E(spl)mbeta-HLH': 0.4692771373356999, 'l(1)10Bb': 0.4668608739627858, 'Ras64B': 0.4572253222285115, 'RpII18': 0.4560754192618993, 'chif': 0.4531320166149416, 'lncRNA:Hsromega': 0.4348974249478839, 'mal': 0.4312591815961508, 'Act42A': 1.0}), gene2occurrence=frozendict.frozendict({}), transcription_factor='Act42A', context=frozenset({'activating', 'weight>75.0%'}), score

In [21]:
modules[1]


Regulon(name='Regulon for Act5C', gene2weight=frozendict.frozendict({'chif': 1.862244838145099, 'Sap-r': 1.7134937978265417, 'inv': 1.648536505862241, 'Mybbp1A': 1.6254871975386755, 'CycA': 1.621240140658466, 'hth': 1.495037858642255, 'Poxm': 1.4406668015333692, 'kkv': 1.385262928056424, 'Hsp67Ba': 1.2257543091651377, 'numb': 1.1888846149543304, 'ifc': 1.1723181001823542, 'RpII215': 1.1571919566646451, 'exd': 0.9099037081364286, 'Lar': 0.8962733511521475, 'sens': 0.7597792426852297, 'okr': 0.6839832913971687, 'l(1)10Bb': 0.6826278894700637, 'Gart': 0.6787329433409397, 'Gli': 0.6493283879687307, 'Hsp27': 0.6424547826875752, 'PpV': 0.5823025276632724, 'cort': 0.5621610033854862, 'bic': 0.5572906209563305, 'shi': 0.5473106732062405, 'Ras64B': 0.5342821512515061, 'hfw': 0.5315908599017927, 'sesB': 0.5233149967354643, 'peb': 0.5034281489411072, 'mago': 0.4732398930554864, 'bsh': 0.4707982703934201, 'E(bx)': 0.44031481432773234, 'ninaA': 0.41890955227139764, 'E(z)': 0.4051692219289528, 'wek'

In [22]:
df = prune2df(dbs, modules, motif_annotations_fname=motif_annotation_file) # Prune modules for targets with cis regulatory footprints (RcisTarget)
df

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Enrichment            \
                                                             AUC       NES   
TF      MotifID                                                              
Myb     swissregulon__sacCer__BAS1                      0.226508  4.181254   
        cisbp__M2083                                    0.189037  3.245416   
        jaspar__MA0278.1                                0.191705  3.312054   
        transfac_pro__M01554                            0.188515  3.232378   
        transfac_pro__M08890                            0.186253  3.175880   
cg      transfac_pro__M06855                            0.169490  3.767730   
D       taipale__SOX8_DBD_AACAATRTKCAGWGTT              0.082784  3.281889   
EcR     flyfactorsurvey__EcR_SANGER_5_FBgn0000546       0.152209  3.279833   
Jra     cisbp__M2979                                    0.163905  3.117649   
        taipale__NFE2_DBD_NATGASTCATN                   0.183737  3.752150   
RpII215 swissregulon__hs__TBP.p2                        0.134176  3.632359   
esg     idmmpmm__sna                                    0.250140  3.188918   
l(1)sc  cisbp__M4745                                    0.158791  3.284826   
        flyfactorsurvey__ase_da_SANGER_10_FBgn0000137   0.154198  3.115810   
opa     flyfactorsurvey__opa_SOLEXA_FBgn0003002         0.139417  3.156266   
        flyfactorsurvey__Opa_SANGER_5_FBgn0003002       0.147383  3.460980   
pho     transfac_pro__M05610                            0.157654  3.580037   
pnt     homer__GGNTCTCGCGAGAAC_ZBTB33                   0.179370  3.144966   
sna     transfac_public__M00184                         0.210229  3.273015   
        transfac_pro__M00693                            0.230304  3.796657   
EcR     flyfactorsurvey__EcR_SANGER_5_FBgn0000546       0.167430  3.790972   
RpII215 swissregulon__hs__TBP.p2                        0.155167  4.159163   
        jaspar__MA0108.2                                0.131305  3.071372   
        cisbp__M4010                                    0.134303  3.208058   
esg     idmmpmm__sna                                    0.241428  3.249657   
l(1)sc  cisbp__M4745                                    0.148302  3.739659   
        flyfactorsurvey__ase_da_SANGER_10_FBgn0000137   0.139116  3.346225   
opa     flyfactorsurvey__opa_SOLEXA_FBgn0003002         0.156845  3.445981   
        flyfactorsurvey__Opa_SANGER_5_FBgn0003002       0.165806  3.765050   
pho     transfac_pro__M05610                            0.156747  3.107349   
pnt     homer__GGNTCTCGCGAGAAC_ZBTB33                   0.195273  3.974945   
        homer__ATTCTCGCGAGA_GFX                         0.170766  3.200240   
        jaspar__MA0527.1                                0.172941  3.269001   
        cisbp__M2323                                    0.180771  3.516540   
        cisbp__M4454                                    0.197651  4.050124   

                                                                             \
                                                      MotifSimilarityQvalue   
TF      MotifID                                                               
Myb     swissregulon__sacCer__BAS1                                 0.000000   
        cisbp__M2083                                               0.000000   
        jaspar__MA0278.1                                           0.000000   
        transfac_pro__M01554                                       0.000000   
        transfac_pro__M08890                                       0.000000   
cg      transfac_pro__M06855                                       0.000000   
D       taipale__SOX8_DBD_AACAATRTKCAGWGTT                         0.000000   
EcR     flyfactorsurvey__EcR_SANGER_5_FBgn0000546                  0.000000   
Jra     cisbp__M2979                                               0.000778   
        taipale__NFE2_DBD_NATGASTCATN                              0.000109   
RpII215 swissregulon__hs__TBP.p2                              

In [23]:
regulons = df2regulons(df) # convert data frame to rergulons
regulons

Create regulons from a dataframe of enriched features.
Additional columns saved: []


[Regulon(name='D(+)', gene2weight=frozendict.frozendict({'ca': 2.2440719936069633, 'Abl': 0.5855768918041399, 'LanA': 0.019288717261368247, 'FER': 0.45695213641712257, 'rod': 0.26654400153325364, 'dlg1': 0.007410874835727238}), gene2occurrence=frozendict.frozendict({}), transcription_factor='D', context=frozenset({'taipale__SOX8_DBD_AACAATRTKCAGWGTT.png', 'activating'}), score=0.7732197168837899, nes=0.0, orthologous_identity=0.0, similarity_qvalue=0.0, annotation=''),
 Regulon(name='EcR(+)', gene2weight=frozendict.frozendict({'pnt': 0.23004627849770867, 'Pka-C1': 0.4313048742777852, 'E(spl)m4-BFM': 0.5910802345091158, 'dsh': 0.004255660334318411, 'E(spl)m5-HLH': 2.5120211208089547, 'rib': 0.1833977334371378, 'Brd': 0.33955544761413853, 'Abl': 0.23366809232288063, 'EcR': 1.0, 'abd-A': 0.4798785326723705, 'S': 0.04141678549124019, 'cv-2': 0.6321239119440264, 'pip': 0.11553120051219401, 'Argk': 0.1482898143169805, 'dor': 1.327384431063267, 'E(spl)malpha-BFM': 3.0236416700968403}), gene2o

In [16]:
auc_ntx = aucell(ex_matrix_sub, regulons, num_workers=1)
auc_ntx

100%|██████████| 13/13 [00:00<00:00, 596.74it/s]


Regulon,Dfd(+),Eno(+),Jra(+),Myb(+),aop(+),cg(+),en(+),gsb-n(+),hb(+),mor(+),opa(+),pnt(+),slp1(+)
Cell,,,,,,,,,,,,,
TP1_AAACCTGAGTAACCCT,0.0,0.0,0.000000,0.040138,0.000000,0.0,0.000000,0.0,0.017390,0.031351,0.000000,0.000744,0.010282
TP1_AAAGATGTCTCCCTGA,0.0,0.0,0.000000,0.005352,0.000000,0.0,0.000000,0.0,0.012693,0.009031,0.000000,0.003995,0.008078
TP1_AAAGCAATCTTCCTTC,0.0,0.0,0.000000,0.018731,0.000000,0.0,0.000000,0.0,0.013270,0.033843,0.000000,0.002686,0.008813
TP1_AAATGCCGTGGGTCAA,0.0,0.0,0.000000,0.008028,0.000000,0.0,0.000000,0.0,0.013270,0.022938,0.000000,0.002480,0.005141
TP1_AACTGGTGTCAACATC,0.0,0.0,0.001654,0.032185,0.001769,0.0,0.000000,0.0,0.029625,0.008429,0.000000,0.003288,0.008813
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TP1_TTAGGACCAAACCTAC,0.0,0.0,0.000000,0.026759,0.000000,0.0,0.000710,0.0,0.020146,0.021820,0.137836,0.003582,0.009547
TP1_TTCGAAGGTTAAGGGC,0.0,0.0,0.000000,0.037462,0.000000,0.0,0.001184,0.0,0.012693,0.033437,0.000000,0.003430,0.008078
TP1_TTCGGTCAGTTGAGTA,0.0,0.0,0.000000,0.029434,0.000000,0.0,0.000237,0.0,0.013270,0.015302,0.000000,0.002865,0.012485


In [ ]:
# TODO: save intermediate products to file - see tutorial https://github.com/aertslab/pySCENIC/blob/master/notebooks/pySCENIC%20-%20Full%20pipeline.ipynb
n = 1  # TODO: decide on n based on computational resources 
all_results = [None] * n
for i in range(0, n):
    """ phase 1 - GRN inference, generation of co-expression modules """
    adjacencies = grnboost2(ex_matrix, tf_names, verbose=True) # adjacencies table of tf, target and importance weight
    modules = list(modules_from_adjacencies(adjacencies, ex_matrix)) # module generation - candidate regulons from TF-target gene interactions 
    """ phase 2+3 - Regulon prediction """
    df = prune2df(dbs, modules, motif_annotations_fname=motif_annotation_file) # Prune modules for targets with cis regulatory footprints (RcisTarget)
    regulons = df2regulons(df) # convert data frame to rergulons
    """ phase 4 - cellular enrichment """
    auc_ntx = aucell(ex_matrix, regulons, num_workers=1)  # Calculate enrichment of gene signatures for single cells.
    all_results[i] = auc_ntx # save for later 

# TODO - keep only modules that appear in over X percent...
# AUCell returns A dataframe with the AUCs (n_cells x n_modules).

    